Experiment and model pruning logic
- determine retention period: (e.g. 10 days)
- detemine minimum to retrain (even if older than expiry period)
- determine if we can know a priori if a model/experiment run can be linked to a deployed model


In [1]:
from dotenv import load_dotenv

# load locally defined environmental variables
load_dotenv(dotenv_path="../env/.env.ae5")  # take environment variables from .env.

True

In [2]:
from src.anaconda.mlflow.service.factory import build_mlflow_client, build_client
from src.anaconda.mlflow.client import AnacondaMlFlowClient

client: AnacondaMlFlowClient = build_client(client=build_mlflow_client())

In [5]:
client.prune(pruneables=client.get_pruneables(), dry_run=False)

Reviewing experiment 1
Reviewing experiment 0
